Single Race Exploratory Data Analysis -

The purpose of this notebook is to explore and wrangle data from the Fast-F1 API for a single race. This serves as a starting point to establish an organized procedure for data acquisition, preparation, and early exploration before scaling to a multi-race analysis. Descriptive statistics and preliminary visualizations will be generated to provide an initial understanding of the data, identify patterns or anomalies, and inform subsequent feature engineering and modeling decisions.

The code below adds the parent directory to Python’s module search path and configures logging to suppress all FastF1 logs below the warning level. This will enable subsequent code blocks that use imports to work seamlessly and keep my resulting code compilations clean and easy to read.

In [1]:
import sys
import os
import logging

root = os.path.abspath("..")
sys.path.append(root)

logging.getLogger('fastf1').setLevel(logging.WARNING)

In this section, I import Python libraries for data visualization, numerical analysis, and working with Pandas dataframes that the FastF1 API is primarily structured with. I also import custom functions and modules for preprocessing F1 data and constants. To support full visibility into the datasets without truncation, I configure Pandas display options to show all rows and columns.

In [2]:
from src.data import f1_data
from src.utils import f1_constants, f1_pandas_helpers
from src.preprocessing import telemetry_cleaning, feature_engineering

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)  # reset_option to compact view
pd.set_option('display.max_columns', None)


The following code initializes a single F1 race session by defining parameters such as year, location, and session type. These values are passed into the custom F1Session class (from f1_data.py), which creates a session object built on top of Fast-F1. This object provides access to race data as well as custom functions I’ve implemented.

The session parameters were chosen to best match Tier 1 control qualities:

- Weather: Abu Dhabi (clear, dry conditions)
- Max Speed: C5 Ultra Soft Tires
- Minimize Outliers: Qualifying Sessions (Q3 & Q2)
- Traffic: Avoid tow laps to avoid slipstream bias

In [3]:
year = 2024
grand_prix = f1_constants.F1Constants.LOCATIONS["Abu Dhabi"]
session_type = f1_constants.F1Constants.SESSIONS["Q"]

session = f1_data.F1Session(year, grand_prix, session_type)

req         WARNING 	DEFAULT CACHE ENABLED! (671.15 MB) /Users/phillipsmith/Library/Caches/fastf1
core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '55', '27', '1', '10', '63', '14', '77', '11', '22', '30', '18', '16', '20', '23', '24', '44', '43', '61']


All drivers who participated in the specific location's Q2 and Q3 sessions will be analyzed and assigned variables to be identified by their three-letter name code. Q3 data will be used for each driver whenever available; otherwise, Q2 data will be selected.

In [4]:
norris = f1_constants.F1Constants.DRIVERS["Lando Norris"]
piastri = f1_constants.F1Constants.DRIVERS["Oscar Piastri"]
verstappen = f1_constants.F1Constants.DRIVERS["Max Verstappen"]
perez = f1_constants.F1Constants.DRIVERS["Sergio Perez"]
sainz = f1_constants.F1Constants.DRIVERS["Carlos Sainz"]
leclerc = f1_constants.F1Constants.DRIVERS["Charles Leclerc"]
bottas = f1_constants.F1Constants.DRIVERS["Valtteri Bottas"]
alonso = f1_constants.F1Constants.DRIVERS["Fernando Alonso"]
stroll = f1_constants.F1Constants.DRIVERS["Lance Stroll"]
gasly = f1_constants.F1Constants.DRIVERS["Pierre Gasly"]
hulkenberg = f1_constants.F1Constants.DRIVERS["Nico Hulkenberg"]
magnussen = f1_constants.F1Constants.DRIVERS["Kevin Magnussen"]
lawson = f1_constants.F1Constants.DRIVERS["Liam Lawson"]
tsunoda = f1_constants.F1Constants.DRIVERS["Yuki Tsunoda"]
russell = f1_constants.F1Constants.DRIVERS["George Russell"]

This function returns filtered telemetry points on or within a circular radius of a given turn.

In [5]:
corner_position = session.get_circuit_info().corners
corner_position_cleaned = telemetry_cleaning.clean_circuit_corner_data(corner_position)

turn = 12
radius = 1500

corner_position_cleaned

,Turn,X (1/10 m),Y (1/10 m),Angle (°),Distance (1/10 m)
0,1,4294.859375,2690.717773,-35.340600,377.851945
1,2,3848.351318,4987.570801,42.198164,626.674566
2,3,2206.551025,6155.120117,-163.169332,834.459836
3,4,2378.656250,8052.805176,-8.298533,1009.587697
4,5,1839.358276,11771.707031,84.326847,1416.310910
5,6,-2150.159424,655.311096,-148.988037,2616.829683
6,7,-1583.492065,587.163330,46.320930,2677.365930
7,8,-823.991882,-1256.985107,-144.588429,2877.258977
8,9,5774.805664,-4917.836914,-21.958205,3682.011370
9,10,3125.774414,-3602.140381,82.890762,4026.195705


For each driver, their fastest qualifying lap telemetry is singled out by filtering by the timestamps for each sector. The resulting sector datasets are cleaned and used to derived additoinal features to identify deeper insights into drivers' response to track dynamics.

Q3 data for Lando Norris, McLaren

In [6]:
q1, q2, q3 = session.get_laps(norris).split_qualifying_sessions()
norris_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:59.496000"
sector_1_end = "0 days 01:17:16.260000"
sector_2_end = "0 days 01:17:52.036000"
sector_3_end = "0 days 01:18:21.897000"

# norris_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_start, end=sector_1_end)
# norris_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector1_telemetry, norris)
# norris_sector1_telemetry_cleaned

# norris_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_end, end=sector_2_end)
# norris_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector2_telemetry, norris)
# norris_sector2_telemetry_cleaned

norris_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_2_end, end=sector_3_end)
norris_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector3_telemetry, norris)
norris_sector3_telemetry_features = feature_engineering.TelemetryFeatures(norris_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()

corner_telemetry = telemetry_cleaning.filter_corner_telemetry(norris_sector3_telemetry_features, corner_position_cleaned, turn, radius)
corner_telemetry

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8466,NOR,0 days 01:18:01.753000,0 days 00:18:22.024000,11213.666652,78.055618,0.000000,-1.052189e+01,7,100.000000,0,25119.320300,1898.000000,-2901.000000,-231.000000
8467,NOR,0 days 01:18:01.918000,0 days 00:18:22.189000,11177.000000,78.055618,-1.736113,3.467986e-12,7,100.000000,0,25132.217778,1789.920274,-2744.696950,-231.628348
8468,NOR,0 days 01:18:01.994000,0 days 00:18:22.265000,11179.375000,77.923673,-1.736112,-7.826737e+01,7,100.000000,0,25138.181231,1747.000000,-2670.000000,-232.000000
8469,NOR,0 days 01:18:02.238000,0 days 00:18:22.509000,11187.000000,77.500062,-20.833350,-3.395976e-13,7,100.000000,0,25157.017778,1668.584786,-2461.177690,-232.785925
8470,NOR,0 days 01:18:02.374000,0 days 00:18:22.645000,10763.360000,74.666726,-20.833350,-1.972855e+01,7,66.000000,0,25166.823079,1646.000000,-2374.000000,-233.000000
8471,NOR,0 days 01:18:02.638000,0 days 00:18:22.909000,9941.000000,69.166722,-26.041687,-1.085070e+01,7,0.000000,1,25184.684444,1619.757216,-2274.194151,-233.027698
8472,NOR,0 days 01:18:02.798000,0 days 00:18:23.069000,9191.000000,65.000052,-27.777800,7.500173e-12,7,0.000000,1,25195.084444,1605.117830,-2187.128248,-232.993946
8473,NOR,0 days 01:18:02.834000,0 days 00:18:23.105000,9171.425000,64.000051,-27.777800,-1.260082e+02,7,0.000000,1,25197.350605,1601.000000,-2158.000000,-233.000000
8474,NOR,0 days 01:18:02.958000,0 days 00:18:23.229000,9104.000000,60.555604,-43.402812,-1.308895e-12,7,0.000000,1,25204.773333,1584.752320,-2032.959172,-233.052032
8475,NOR,0 days 01:18:03.034000,0 days 00:18:23.305000,9266.925000,57.256990,-43.402813,8.965171e+01,7,0.000000,1,25208.817108,1577.000000,-1969.000000,-233.000000


Exploratory Data Analysis - Lando Norris

In [7]:
norris_eda_summary = f1_pandas_helpers.get_driver_eda_summary(norris_sector3_telemetry_features, norris)
norris_eda_summary

--- EDA Summary for NOR ---
Row Count: 231

Speed (m/s) -->
Max   : 88.336380
Mean  : 56.959762
Median: 57.500046
StdDev: 14.514933

Acceleration (m/s²) -->
Max   : 16.666680
Mean  : -0.228561
Median: 2.314817
StdDev: 11.157616

nGear -->
Shifts: 13

Throttle (%) -->
Events: 5
Mean  : 65.951264
StdDev: 40.945685

BrakesApplied -->
Events: 3



Q3 data for Oscar Piastri, McLaren

In [8]:
q1, q2, q3 = session.get_laps(piastri).split_qualifying_sessions()
piastri_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:27.506000"
sector_1_end = "0 days 01:16:44.333000"
sector_2_end = "0 days 01:17:20.186000"
sector_3_end = "0 days 01:17:50.110000"

# piastri_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_start, end=sector_1_end)
# piastri_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector1_telemetry, piastri)
# piastri_sector1_telemetry_cleaned

# piastri_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_end, end=sector_2_end)
# piastri_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector2_telemetry, piastri)
# piastri_sector2_telemetry_cleaned

piastri_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_2_end, end=sector_3_end)
piastri_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector3_telemetry, piastri)
piastri_sector3_telemetry_features = feature_engineering.TelemetryFeatures(piastri_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
piastri_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8159,PIA,0 days 01:17:20.195000,0 days 00:17:43.940000,11072.939722,87.805624,-13.888878,-2.201900e+02,8,40.999936,0,24488.931434,4469.000000,-5127.000000,-230.000000
8160,PIA,0 days 01:17:20.277000,0 days 00:17:44.022000,10895.000000,86.666736,-31.944457,-3.857784e-04,8,0.000000,1,24495.916944,4518.342601,-5146.361327,-229.949366
8161,PIA,0 days 01:17:20.434000,0 days 00:17:44.179000,10455.400179,81.651456,-31.944518,-1.453478e+02,8,0.000000,1,24508.637455,4580.000000,-5171.000000,-230.000000
8162,PIA,0 days 01:17:20.477000,0 days 00:17:44.222000,10335.000000,80.277842,-38.194475,-6.422213e-12,8,0.000000,1,24511.972500,4606.842154,-5181.564809,-229.847388
8163,PIA,0 days 01:17:20.633000,0 days 00:17:44.378000,10959.000000,74.319504,-38.194475,4.726084e+03,8,0.000000,1,24523.551990,4728.000000,-5229.000000,-229.000000
8164,PIA,0 days 01:17:20.637000,0 days 00:17:44.382000,10975.000000,74.166726,-19.290139,-1.505387e-14,8,0.000000,1,24523.839167,4730.986183,-5230.169971,-228.985219
8165,PIA,0 days 01:17:20.873000,0 days 00:17:44.618000,10992.044444,69.614253,-19.290139,-2.275423e+01,8,0.000000,1,24539.985819,4888.000000,-5291.000000,-229.000000
8166,PIA,0 days 01:17:20.997000,0 days 00:17:44.742000,11001.000000,67.222276,-22.111664,-4.355642e-04,8,0.000000,1,24548.039167,4963.105829,-5317.491724,-229.688660
8167,PIA,0 days 01:17:21.054000,0 days 00:17:44.799000,11164.910316,65.961911,-22.111689,4.504636e+01,8,0.000000,1,24551.672515,4999.000000,-5327.000000,-230.000000
8168,PIA,0 days 01:17:21.198000,0 days 00:17:44.943000,11579.000000,62.777828,-15.625013,1.085070e+01,6,0.000000,1,24560.657500,5097.931768,-5341.042169,-230.497844


Exploratory Data Analysis - Oscar Piastri

In [9]:
piastri_eda_summary = f1_pandas_helpers.get_driver_eda_summary(piastri_sector3_telemetry_features, piastri)
piastri_eda_summary

--- EDA Summary for PIA ---
Row Count: 240

Speed (m/s) -->
Max   : 87.805624
Mean  : 57.314188
Median: 57.638935
StdDev: 14.235765

Acceleration (m/s²) -->
Max   : 15.046308
Mean  : -0.864437
Median: 2.314815
StdDev: 11.718347

nGear -->
Shifts: 15

Throttle (%) -->
Events: 4
Mean  : 64.000187
StdDev: 41.025113

BrakesApplied -->
Events: 3



Q3 data for Max Verstappen, Red Bull

In [10]:
q1, q2, q3 = session.get_laps(verstappen).split_qualifying_sessions()
verstappen_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:08:27.031000"
sector_1_end = "0 days 01:08:44.036000"
sector_2_end = "0 days 01:09:19.766000"
sector_3_end = "0 days 01:09:49.959000"

# verstappen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_start, end=sector_1_end)
# verstappen_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector1_telemetry, verstappen)
# verstappen_sector1_telemetry_cleaned

# verstappen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_end, end=sector_2_end)
# verstappen_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector2_telemetry, verstappen)
# verstappen_sector2_telemetry_cleaned

verstappen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_2_end, end=sector_3_end)
verstappen_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector3_telemetry, verstappen)
verstappen_sector3_telemetry_features = feature_engineering.TelemetryFeatures(verstappen_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
verstappen_sector3_telemetry_features


,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
9139,VER,0 days 01:09:19.874000,0 days 00:20:15.171000,11422.871047,89.847743,-4.978111,-367.211217,8,29.032320,0,8822.800856,4434.000000,-5114.000000,-230.000000
9140,VER,0 days 01:09:19.955000,0 days 00:20:15.252000,11386.000000,89.444516,-34.722220,-0.000440,8,0.000000,1,8829.808333,4511.135071,-5143.983356,-229.672334
9141,VER,0 days 01:09:20.174000,0 days 00:20:15.471000,10698.203728,81.840350,-34.722316,55.006547,8,0.000000,1,8847.284928,4726.000000,-5228.000000,-229.000000
9142,VER,0 days 01:09:20.275000,0 days 00:20:15.572000,10381.000000,78.333396,-29.166655,-0.001081,8,0.000000,1,8854.875000,4782.499755,-5250.769217,-228.851023
9143,VER,0 days 01:09:20.434000,0 days 00:20:15.731000,10351.585036,73.695898,-29.166827,330.288609,8,0.000000,1,8866.443058,4849.000000,-5277.000000,-229.000000
9144,VER,0 days 01:09:20.475000,0 days 00:20:15.772000,10344.000000,72.500058,-15.624994,-0.018987,8,0.000000,1,8869.375000,4872.320023,-5285.526869,-229.160848
9145,VER,0 days 01:09:20.634000,0 days 00:20:15.931000,10633.180901,70.015684,-15.628013,-10992.379164,8,0.000000,1,8880.507433,4976.000000,-5321.000000,-230.000000
9146,VER,0 days 01:09:20.635000,0 days 00:20:15.932000,10635.000000,70.000056,-26.620392,34.078015,7,0.000000,1,8880.575000,4976.608033,-5321.190491,-230.004011
9147,VER,0 days 01:09:20.875000,0 days 00:20:16.172000,11236.000000,63.611162,-18.441668,-0.000413,5,0.000000,1,8895.841667,5101.739971,-5350.009412,-230.417191
9148,VER,0 days 01:09:20.994000,0 days 00:20:16.291000,11066.635083,61.416603,-18.441717,65.779174,5,0.000000,1,8902.976243,5182.000000,-5345.000000,-231.000000


Exploratory Data Analysis - Max Verstappen

In [11]:
verstappen_eda_summary = f1_pandas_helpers.get_driver_eda_summary(verstappen_sector3_telemetry_features, verstappen)
verstappen_eda_summary

--- EDA Summary for VER ---
Row Count: 232

Speed (m/s) -->
Max   : 89.847743
Mean  : 56.024903
Median: 54.382334
StdDev: 14.761748

Acceleration (m/s²) -->
Max   : 18.978646
Mean  : -0.903044
Median: 2.420398
StdDev: 11.482968

nGear -->
Shifts: 13

Throttle (%) -->
Events: 6
Mean  : 65.042522
StdDev: 41.857983

BrakesApplied -->
Events: 3



Q3 data for Sergio Perez, Red Bull

In [12]:
q1, q2, q3 = session.get_laps(perez).split_qualifying_sessions()
perez_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:14:50.040000"
sector_1_end = "0 days 01:15:06.945000"
sector_2_end = "0 days 01:15:42.890000"
sector_3_end = "0 days 01:16:13.127000"

# perez_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_start, end=sector_1_end)
# perez_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector1_telemetry, perez)
# perez_sector1_telemetry_cleaned

# perez_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_end, end=sector_2_end)
# perez_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector2_telemetry, perez)
# perez_sector2_telemetry_cleaned

perez_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_2_end, end=sector_3_end)
perez_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector3_telemetry, perez)
perez_sector3_telemetry_features = feature_engineering.TelemetryFeatures(perez_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
perez_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7372,PER,0 days 01:15:42.933000,0 days 00:16:01.230000,11414.437311,89.114896,-9.285057,-2.329101e+02,8,40.111413,0,24513.357526,4501.000000,-5140.000000,-230.000000
7373,PER,0 days 01:15:43.077000,0 days 00:16:01.374000,11329.000000,87.777848,-42.824108,6.848041e+01,8,0.000000,1,24525.578333,4612.215407,-5183.090058,-229.497804
7374,PER,0 days 01:15:43.317000,0 days 00:16:01.614000,11028.000000,77.500062,-26.388811,-6.386788e-03,8,0.000000,1,24544.178333,4721.729450,-5225.930630,-229.023316
7375,PER,0 days 01:15:43.334000,0 days 00:16:01.631000,10994.765125,77.051452,-26.388919,5.881916e+01,8,0.000000,1,24545.436085,4732.000000,-5230.000000,-229.000000
7376,PER,0 days 01:15:43.517000,0 days 00:16:01.814000,10637.000000,72.222280,-15.625013,6.106227e-12,7,0.000000,1,24558.622778,4860.207552,-5280.800769,-228.955249
7377,PER,0 days 01:15:43.533000,0 days 00:16:01.830000,10646.100000,71.972280,-15.625013,-5.224385e+01,7,0.000000,1,24559.757122,4871.000000,-5285.000000,-229.000000
7378,PER,0 days 01:15:43.677000,0 days 00:16:01.974000,10728.000000,69.722278,-23.148127,-1.279406e-03,7,0.000000,1,24569.778333,4961.688303,-5317.586467,-229.788617
7379,PER,0 days 01:15:43.714000,0 days 00:16:02.011000,10838.999808,68.865797,-23.148174,5.416457e+01,7,0.000000,1,24572.248845,4988.000000,-5324.000000,-230.000000
7380,PER,0 days 01:15:43.917000,0 days 00:16:02.214000,11448.000000,64.166718,-12.152766,-7.390389e-04,7,0.000000,1,24585.178333,5159.937946,-5343.752580,-230.858148
7381,PER,0 days 01:15:43.954000,0 days 00:16:02.251000,11445.687504,63.717066,-12.152794,-7.057363e+01,7,0.000000,1,24587.502640,5186.000000,-5345.000000,-231.000000


Exploratory Data Analysis - Sergio Perez

In [13]:
perez_eda_summary = f1_pandas_helpers.get_driver_eda_summary(perez_sector3_telemetry_features, perez)
perez_eda_summary

--- EDA Summary for PER ---
Row Count: 233

Speed (m/s) -->
Max   : 89.114896
Mean  : 56.033409
Median: 56.111156
StdDev: 14.932516

Acceleration (m/s²) -->
Max   : 13.889196
Mean  : -0.848806
Median: 2.314817
StdDev: 11.872458

nGear -->
Shifts: 13

Throttle (%) -->
Events: 7
Mean  : 64.284898
StdDev: 43.065403

BrakesApplied -->
Events: 3



Q3 data for Carlos Sainz, Ferrari

In [14]:
q1, q2, q3 = session.get_laps(sainz).split_qualifying_sessions()
sainz_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:53.219000"
sector_1_end = "0 days 01:17:10.014000"
sector_2_end = "0 days 01:17:45.801000"
sector_3_end = "0 days 01:18:15.858000"

# sainz_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_start, end=sector_1_end)
# sainz_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector1_telemetry, sainz)
# sainz_sector1_telemetry_cleaned

# sainz_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_end, end=sector_2_end)
# sainz_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector2_telemetry, sainz)
# sainz_sector2_telemetry_cleaned

sainz_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_2_end, end=sector_3_end)
sainz_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector3_telemetry, sainz)
sainz_sector3_telemetry_features = feature_engineering.TelemetryFeatures(sainz_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
sainz_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7969,SAI,0 days 01:17:45.815000,0 days 00:17:15.181000,11403.605068,90.045906,-2.083347,-1.294193e+03,8,13.060029,0,24539.332360,4339.000000,-5077.000000,-230.000000
7970,SAI,0 days 01:17:45.837000,0 days 00:17:15.203000,11392.000000,90.000072,-30.555600,9.976314e-03,8,8.000000,1,24541.243333,4363.610797,-5086.554693,-229.951838
7971,SAI,0 days 01:17:46.035000,0 days 00:17:15.401000,10259.439268,83.950063,-30.553624,-3.473203e+03,8,0.079995,1,24557.860336,4602.000000,-5179.000000,-230.000000
7972,SAI,0 days 01:17:46.037000,0 days 00:17:15.403000,10248.000000,83.888956,-37.500030,3.372993e+01,8,0.000000,1,24558.021111,4603.712616,-5179.666685,-229.997587
7973,SAI,0 days 01:17:46.237000,0 days 00:17:15.603000,9595.000000,76.388950,-30.754043,8.968688e-04,8,0.000000,1,24573.298889,4686.202854,-5212.268178,-229.273409
7974,SAI,0 days 01:17:46.315000,0 days 00:17:15.681000,9961.043458,73.990135,-30.753973,8.349039e+01,8,0.000000,1,24578.822426,4713.000000,-5223.000000,-229.000000
7975,SAI,0 days 01:17:46.517000,0 days 00:17:15.883000,10909.000000,67.777832,-13.888915,2.412396e-04,8,0.000000,1,24592.276667,4833.233344,-5270.510924,-228.749108
7976,SAI,0 days 01:17:46.635000,0 days 00:17:16.001000,11272.956645,66.138940,-13.888887,-1.556266e-03,8,0.000000,1,24599.897976,4909.000000,-5299.000000,-229.000000
7977,SAI,0 days 01:17:46.834000,0 days 00:17:16.200000,11886.746678,63.375052,-13.889196,-1.388791e+03,8,0.000000,1,24612.359888,5025.000000,-5333.000000,-230.000000
7978,SAI,0 days 01:17:46.837000,0 days 00:17:16.203000,11896.000000,63.333384,-18.055570,2.777780e+01,7,0.000000,1,24612.543333,5027.066644,-5333.344195,-230.013543


Exploratory Data Analysis - Carlos Sainz

In [15]:
sainz_eda_summary = f1_pandas_helpers.get_driver_eda_summary(sainz_sector3_telemetry_features, sainz)
sainz_eda_summary

--- EDA Summary for SAI ---
Row Count: 232

Speed (m/s) -->
Max   : 90.045906
Mean  : 56.295798
Median: 56.090902
StdDev: 15.027524

Acceleration (m/s²) -->
Max   : 17.857162
Mean  : -0.689582
Median: 2.314818
StdDev: 11.860875

nGear -->
Shifts: 14

Throttle (%) -->
Events: 4
Mean  : 66.204831
StdDev: 39.497185

BrakesApplied -->
Events: 3



Q2 data for Charles Leclerc, Ferrari

In [16]:
q1, q2, q3 = session.get_laps(leclerc).split_qualifying_sessions()
leclerc_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:03.473000"
sector_1_end = "0 days 00:57:20.332000"
sector_2_end = "0 days 00:57:56.296000"
sector_3_end = "0 days 00:58:26.338000"

# leclerc_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_start, end=sector_1_end)
# leclerc_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector1_telemetry, leclerc)
# leclerc_sector1_telemetry_cleaned

# leclerc_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_end, end=sector_2_end)
# leclerc_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector2_telemetry, leclerc)
# leclerc_sector2_telemetry_cleaned

leclerc_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_2_end, end=sector_3_end)
leclerc_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector3_telemetry, leclerc)
leclerc_sector3_telemetry_features = feature_engineering.TelemetryFeatures(leclerc_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
leclerc_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
10357,LEC,0 days 00:57:56.313000,0 days 00:22:57.462000,10994.333333,84.074141,-22.376561,-8.634634e+01,8,35.000000,0,24485.250805,4434.000000,-5113.000000,-230.000000
10358,LEC,0 days 00:57:56.433000,0 days 00:22:57.582000,10779.000000,81.388954,-32.738121,3.795660e+01,8,3.000000,1,24494.582500,4552.167108,-5159.196784,-229.514313
10359,LEC,0 days 00:57:56.713000,0 days 00:22:57.862000,10523.000000,72.222280,-22.110273,-1.415148e+00,8,2.000000,1,24514.804722,4759.434144,-5241.776188,-229.001597
10360,LEC,0 days 00:57:56.714000,0 days 00:22:57.863000,10522.840806,72.200170,-22.111689,3.532690e+01,8,2.000000,1,24514.874293,4760.000000,-5242.000000,-229.000000
10361,LEC,0 days 00:57:56.914000,0 days 00:22:58.063000,10491.000000,67.777832,-15.046308,6.661338e-14,8,2.000000,1,24528.428056,4848.553327,-5276.563254,-228.693326
10362,LEC,0 days 00:57:57.074000,0 days 00:22:58.223000,10833.666667,65.370423,-15.046308,-6.276461e-13,8,1.666667,1,24538.544716,4913.000000,-5300.000000,-229.000000
10363,LEC,0 days 00:57:57.374000,0 days 00:22:58.523000,11476.166667,60.856530,-15.046308,4.050929e+02,8,1.041667,1,24556.315184,5085.000000,-5343.000000,-231.000000
10364,LEC,0 days 00:57:57.394000,0 days 00:22:58.543000,11519.000000,60.555604,-6.944450,-9.436896e-14,5,1.000000,1,24557.494722,5100.350536,-5343.831503,-231.034311
10365,LEC,0 days 00:57:57.554000,0 days 00:22:58.703000,11225.285714,59.444492,-6.944450,-3.472225e+01,5,4.428571,1,24566.931511,5230.000000,-5340.000000,-231.000000
10366,LEC,0 days 00:57:57.674000,0 days 00:22:58.823000,11005.000000,58.611158,-11.111120,1.587300e+01,5,7.000000,1,24573.905833,5301.661656,-5330.356077,-231.690289


Exploratory Data Analysis - Charles Leclerc

In [17]:
leclerc_eda_summary = f1_pandas_helpers.get_driver_eda_summary(leclerc_sector3_telemetry_features, leclerc)
leclerc_eda_summary

--- EDA Summary for LEC ---
Row Count: 232

Speed (m/s) -->
Max   : 84.074141
Mean  : 57.125261
Median: 56.567506
StdDev: 14.243424

Acceleration (m/s²) -->
Max   : 19.097237
Mean  : -0.257363
Median: 2.777780
StdDev: 11.649045

nGear -->
Shifts: 13

Throttle (%) -->
Events: 1
Mean  : 70.390000
StdDev: 36.223758

BrakesApplied -->
Events: 3



Q3 data for Valtteri Bottas, Alfa Romeo

In [18]:
q1, q2, q3 = session.get_laps(bottas).split_qualifying_sessions()
bottas_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:15:46.639000"
sector_1_end = "0 days 01:16:03.650000"
sector_2_end = "0 days 01:16:39.552000"
sector_3_end = "0 days 01:17:09.702000"

# bottas_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_start, end=sector_1_end)
# bottas_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector1_telemetry, bottas)
# bottas_sector1_telemetry_cleaned

# bottas_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_end, end=sector_2_end)
# bottas_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector2_telemetry, bottas)
# bottas_sector2_telemetry_cleaned

bottas_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_2_end, end=sector_3_end)
bottas_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector3_telemetry, bottas)
bottas_sector3_telemetry_features = feature_engineering.TelemetryFeatures(bottas_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
bottas_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7153,BOT,0 days 01:16:39.597000,0 days 00:15:27.992000,11403.000000,89.722294,-33.333351,-9.320405e-05,8,100.000000,1,8838.038056,4382.438363,-5093.401159,-230.821314
7154,BOT,0 days 01:16:39.834000,0 days 00:15:28.229000,10628.010209,81.822290,-33.333373,2.799765e+01,8,40.750016,1,8856.767338,4587.000000,-5173.000000,-230.000000
7155,BOT,0 days 01:16:39.997000,0 days 00:15:28.392000,10095.000000,76.388950,-28.769756,-8.166376e-03,8,0.000000,1,8868.593611,4701.742126,-5218.646108,-229.059312
7156,BOT,0 days 01:16:40.014000,0 days 00:15:28.409000,10065.371540,75.899864,-28.769895,4.167258e-04,8,0.000000,1,8869.800127,4718.000000,-5225.000000,-229.000000
7157,BOT,0 days 01:16:40.195000,0 days 00:15:28.590000,9749.914063,70.692513,-28.769819,1.156060e+02,8,0.000000,1,8882.315350,4929.000000,-5306.000000,-229.000000
7158,BOT,0 days 01:16:40.277000,0 days 00:15:28.672000,9607.000000,68.333388,-19.290128,-1.317184e-04,8,0.000000,1,8887.726944,4982.198106,-5325.564266,-229.205183
7159,BOT,0 days 01:16:40.495000,0 days 00:15:28.890000,9855.277632,64.128140,-19.290156,7.471551e+01,8,0.000000,1,8901.255113,5024.000000,-5333.000000,-230.000000
7160,BOT,0 days 01:16:40.637000,0 days 00:15:29.032000,10017.000000,61.388938,-8.680554,-4.243234e-04,6,0.000000,1,8909.826944,5089.582398,-5342.716869,-230.413274
7161,BOT,0 days 01:16:40.775000,0 days 00:15:29.170000,10656.111907,60.191021,-8.680613,-7.417912e+02,6,0.000000,1,8918.146008,5179.000000,-5345.000000,-231.000000
7162,BOT,0 days 01:16:40.797000,0 days 00:15:29.192000,10758.000000,60.000048,-25.000020,8.159736e+01,5,0.000000,1,8919.426944,5194.533742,-5343.488729,-231.135766


Exploratory Data Analysis - Valtteri Bottas

In [19]:
bottas_eda_summary = f1_pandas_helpers.get_driver_eda_summary(bottas_sector3_telemetry_features, bottas)
bottas_eda_summary

--- EDA Summary for BOT ---
Row Count: 232

Speed (m/s) -->
Max   : 89.722294
Mean  : 56.195925
Median: 56.655114
StdDev: 14.644661

Acceleration (m/s²) -->
Max   : 14.983879
Mean  : -0.893984
Median: 1.984129
StdDev: 13.216105

nGear -->
Shifts: 12

Throttle (%) -->
Events: 6
Mean  : 66.116727
StdDev: 42.139807

BrakesApplied -->
Events: 4



Q3 data for Fernando Alonso, Aston Martin

In [20]:
q1, q2, q3 = session.get_laps(alonso).split_qualifying_sessions()
alonso_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:45.371000"
sector_1_end = "0 days 01:17:02.191000"
sector_2_end = "0 days 01:17:38.197000"
sector_3_end = "0 days 01:18:08.377000"

# alonso_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_start, end=sector_1_end)
# alonso_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector1_telemetry, alonso)
# alonso_sector1_telemetry_cleaned

# alonso_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_end, end=sector_2_end)
# alonso_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector2_telemetry, alonso)
# alonso_sector2_telemetry_cleaned

alonso_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_2_end, end=sector_3_end)
alonso_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector3_telemetry, alonso)
alonso_sector3_telemetry_features = feature_engineering.TelemetryFeatures(alonso_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
alonso_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8058,ALO,0 days 01:17:38.415000,0 days 00:17:28.949000,11263.912646,82.534792,-28.935225,2.606788e+01,8,45.787526,0,24512.859471,4642.000000,-5195.000000,-230.000000
8059,ALO,0 days 01:17:38.637000,0 days 00:17:29.171000,11010.000000,76.111172,-23.148156,-1.274245e-04,8,0.000000,1,24528.825556,4823.405274,-5264.804026,-229.903323
8060,ALO,0 days 01:17:38.774000,0 days 00:17:29.308000,10864.627846,72.939875,-23.148173,1.038035e+02,8,0.000000,1,24538.217757,4903.000000,-5297.000000,-230.000000
8061,ALO,0 days 01:17:38.997000,0 days 00:17:29.531000,10628.000000,67.777832,0.000000,-1.405414e+02,7,0.000000,1,24553.225556,4980.974317,-5334.587965,-230.257413
8062,ALO,0 days 01:17:39.237000,0 days 00:17:29.771000,11159.000000,67.777832,-33.729946,-1.392817e-02,6,0.000000,1,24569.492222,5092.395603,-5346.682615,-230.921247
8063,ALO,0 days 01:17:39.255000,0 days 00:17:29.789000,11167.549939,67.170693,-33.730197,-1.176768e-04,6,0.000000,1,24570.629074,5107.000000,-5346.000000,-231.000000
8064,ALO,0 days 01:17:39.454000,0 days 00:17:29.988000,11262.074970,60.458384,-33.730220,3.936769e+02,6,0.000000,1,24582.266242,5292.000000,-5328.000000,-232.000000
8065,ALO,0 days 01:17:39.517000,0 days 00:17:30.051000,11292.000000,58.333380,-8.928576,-4.967179e-05,5,0.000000,1,24585.825556,5325.685528,-5322.530977,-232.229396
8066,ALO,0 days 01:17:39.714000,0 days 00:17:30.248000,10799.500160,56.574451,-8.928585,-8.067984e+01,5,0.000000,1,24596.902268,5370.000000,-5303.000000,-233.000000
8067,ALO,0 days 01:17:39.797000,0 days 00:17:30.331000,10592.000000,55.833378,-15.625013,5.425344e+01,5,0.000000,1,24601.458889,5401.914418,-5287.656220,-233.723193


Exploratory Data Analysis - Fernando Alonso

In [21]:
alonso_eda_summary = f1_pandas_helpers.get_driver_eda_summary(alonso_sector3_telemetry_features, alonso)
alonso_eda_summary

--- EDA Summary for ALO ---
Row Count: 234

Speed (m/s) -->
Max   : 82.534792
Mean  : 56.493144
Median: 56.574285
StdDev: 14.141441

Acceleration (m/s²) -->
Max   : 24.305575
Mean  : -0.260180
Median: 2.777780
StdDev: 12.002734

nGear -->
Shifts: 19

Throttle (%) -->
Events: 7
Mean  : 66.841662
StdDev: 40.293347

BrakesApplied -->
Events: 5



Q2 data for Lance Stroll, Aston Martin

In [22]:
q1, q2, q3 = session.get_laps(stroll).split_qualifying_sessions()
stroll_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:56:38.573000"
sector_1_end = "0 days 00:56:55.483000"
sector_2_end = "0 days 00:57:31.468000"
sector_3_end = "0 days 00:58:02.147000"

# stroll_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_start, end=sector_1_end)
# stroll_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector1_telemetry, stroll)
# stroll_sector1_telemetry_cleaned

# stroll_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_end, end=sector_2_end)
# stroll_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector2_telemetry, stroll)
# stroll_sector2_telemetry_cleaned

stroll_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_2_end, end=sector_3_end)
stroll_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector3_telemetry, stroll)
stroll_sector3_telemetry_features = feature_engineering.TelemetryFeatures(stroll_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
stroll_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
10308,STR,0 days 00:57:31.514000,0 days 00:22:50.133000,11358.000000,89.444516,-18.055585,3.060424e-04,8,99.000000,0,24486.523611,4423.488090,-5109.886079,-230.131053
10309,STR,0 days 00:57:31.593000,0 days 00:22:50.212000,11069.254766,88.018125,-18.055560,-1.606981e+02,8,59.894968,0,24493.423613,4470.000000,-5128.000000,-230.000000
10310,STR,0 days 00:57:31.714000,0 days 00:22:50.333000,10627.000000,85.833402,-37.500030,-1.421085e-12,8,0.000000,1,24503.690278,4561.127426,-5163.222857,-230.065185
10311,STR,0 days 00:57:31.814000,0 days 00:22:50.433000,10270.500000,82.083399,-37.500030,1.729799e+02,8,0.000000,1,24511.707972,4640.000000,-5194.000000,-230.000000
10312,STR,0 days 00:57:31.914000,0 days 00:22:50.533000,9914.000000,78.333396,-20.202036,1.125026e-12,8,0.000000,1,24519.356944,4710.897222,-5222.326989,-229.549540
10313,STR,0 days 00:57:32.034000,0 days 00:22:50.653000,10157.818182,75.909152,-20.202036,-1.972855e+00,8,0.818182,1,24528.126956,4794.000000,-5255.000000,-229.000000
10314,STR,0 days 00:57:32.354000,0 days 00:22:50.973000,10808.000000,69.444500,-20.833350,-3.472225e+01,6,3.000000,1,24549.912500,5032.000000,-5335.000000,-230.000000
10315,STR,0 days 00:57:32.554000,0 days 00:22:51.173000,10808.000000,65.277830,-27.777800,1.110223e-12,6,3.000000,1,24562.968056,5124.591233,-5347.963631,-230.764543
10316,STR,0 days 00:57:32.634000,0 days 00:22:51.253000,11018.000000,63.055606,-27.777800,2.170141e+02,6,2.500000,1,24567.891911,5161.000000,-5347.000000,-231.000000
10317,STR,0 days 00:57:32.714000,0 days 00:22:51.333000,11228.000000,60.833382,-10.416675,-3.410221e+01,6,2.000000,1,24572.701389,5201.949895,-5343.646881,-231.189937


Exploratory Data Analysis - Lance Stroll

In [23]:
stroll_eda_summary = f1_pandas_helpers.get_driver_eda_summary(stroll_sector3_telemetry_features, stroll)
stroll_eda_summary

--- EDA Summary for STR ---
Row Count: 224

Speed (m/s) -->
Max   : 89.444516
Mean  : 55.036277
Median: 53.959534
StdDev: 15.047620

Acceleration (m/s²) -->
Max   : 19.097302
Mean  : -0.870515
Median: 2.777779
StdDev: 13.659768

nGear -->
Shifts: 16

Throttle (%) -->
Events: 9
Mean  : 64.191492
StdDev: 42.555919

BrakesApplied -->
Events: 5



Q3 data for Pierre Gasly, Alpine

In [24]:
q1, q2, q3 = session.get_laps(gasly).split_qualifying_sessions()
gasly_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:18.040000"
sector_1_end = "0 days 01:16:34.989000"
sector_2_end = "0 days 01:17:10.860000"
sector_3_end = "0 days 01:17:40.892000"

# gasly_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_start, end=sector_1_end)
# gasly_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector1_telemetry, gasly)
# gasly_sector1_telemetry_cleaned

# gasly_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_end, end=sector_2_end)
# gasly_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector2_telemetry, gasly)
# gasly_sector2_telemetry_cleaned

gasly_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_2_end, end=sector_3_end)
gasly_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector3_telemetry, gasly)
gasly_sector3_telemetry_features = feature_engineering.TelemetryFeatures(gasly_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
gasly_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8239,GAS,0 days 01:17:10.917000,0 days 00:17:53.777000,10847.000000,89.444516,-35.714314,4.487941e+01,8,34.000000,1,24523.701111,4557.954288,-5161.833702,-229.774863
8240,GAS,0 days 01:17:11.197000,0 days 00:17:54.057000,11158.000000,79.444508,-23.148080,-5.517027e-03,8,0.000000,1,24545.945556,4750.881844,-5238.400525,-229.036117
8241,GAS,0 days 01:17:11.214000,0 days 00:17:54.074000,11173.866607,79.050991,-23.148173,1.816563e+01,8,0.000000,1,24547.232508,4760.000000,-5242.000000,-229.000000
8242,GAS,0 days 01:17:11.437000,0 days 00:17:54.297000,11382.000000,73.888948,-19.097238,3.255244e+01,7,0.000000,1,24563.678889,4857.174371,-5279.773111,-228.611191
8243,GAS,0 days 01:17:11.597000,0 days 00:17:54.457000,10916.000000,70.833390,-13.888848,-3.310216e-03,6,0.000000,1,24575.012222,4924.031264,-5304.181098,-228.917257
8244,GAS,0 days 01:17:11.614000,0 days 00:17:54.474000,10974.720612,70.597280,-13.888904,-5.190150e+01,6,0.000000,1,24576.195233,4932.000000,-5307.000000,-229.000000
8245,GAS,0 days 01:17:11.837000,0 days 00:17:54.697000,11745.000000,67.500054,-25.462939,-1.190381e-03,5,0.000000,1,24591.212222,5060.322094,-5342.098165,-230.739407
8246,GAS,0 days 01:17:11.874000,0 days 00:17:54.734000,11676.550118,66.557925,-25.462983,-3.936307e-04,5,0.000000,1,24593.564600,5091.000000,-5344.000000,-231.000000
8247,GAS,0 days 01:17:12.054000,0 days 00:17:54.914000,11343.550118,61.974588,-25.463054,9.259297e+02,5,0.000000,1,24604.548450,5266.000000,-5334.000000,-232.000000
8248,GAS,0 days 01:17:12.077000,0 days 00:17:54.937000,11301.000000,61.388938,-4.166670,-4.166667e+01,5,0.000000,1,24605.945556,5283.733308,-5331.877517,-232.148744


Exploratory Data Analysis - Pierre Gasly

In [25]:
gasly_eda_summary = f1_pandas_helpers.get_driver_eda_summary(gasly_sector3_telemetry_features, gasly)
gasly_eda_summary

--- EDA Summary for GAS ---
Row Count: 232

Speed (m/s) -->
Max   : 89.444516
Mean  : 56.669368
Median: 56.833379
StdDev: 14.407951

Acceleration (m/s²) -->
Max   : 19.675942
Mean  : -1.475729
Median: 2.083335
StdDev: 12.647495

nGear -->
Shifts: 18

Throttle (%) -->
Events: 6
Mean  : 65.345836
StdDev: 42.922634

BrakesApplied -->
Events: 4



Q3 data for Nico Hulkenberg, Haas

In [26]:
q1, q2, q3 = session.get_laps(hulkenberg).split_qualifying_sessions()
hulkenberg_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:03.696000"
sector_1_end = "0 days 01:16:20.651000"
sector_2_end = "0 days 01:16:56.369000"
sector_3_end = "0 days 01:17:26.431000"

# hulkenberg_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_start, end=sector_1_end)
# hulkenberg_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector1_telemetry, hulkenberg)
# hulkenberg_sector1_telemetry_cleaned

# hulkenberg_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_end, end=sector_2_end)
# hulkenberg_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector2_telemetry, hulkenberg)
# hulkenberg_sector2_telemetry_cleaned

hulkenberg_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_2_end, end=sector_3_end)
hulkenberg_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector3_telemetry, hulkenberg)
hulkenberg_sector3_telemetry_features = feature_engineering.TelemetryFeatures(hulkenberg_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
hulkenberg_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8314,HUL,0 days 01:16:56.478000,0 days 00:18:05.455000,11017.000000,87.500070,-30.555580,2.220446e-13,8,7.000000,1,24564.374444,4544.599463,-5156.491772,-230.000724
8315,HUL,0 days 01:16:56.574000,0 days 00:18:05.551000,10674.280000,84.566734,-30.555580,-1.335471e+01,8,3.640000,1,24572.339426,4666.000000,-5204.000000,-230.000000
8316,HUL,0 days 01:16:56.678000,0 days 00:18:05.655000,10303.000000,81.388954,-31.944470,3.781353e+01,8,0.000000,1,24580.652222,4760.261671,-5241.522178,-229.813185
8317,HUL,0 days 01:16:56.878000,0 days 00:18:05.855000,10232.000000,75.000060,-24.381763,-6.472676e-05,8,0.000000,1,24595.652222,4839.886519,-5275.313556,-228.988015
8318,HUL,0 days 01:16:57.074000,0 days 00:18:06.051000,10653.492078,70.221234,-24.381776,2.078363e+01,8,0.000000,1,24609.117156,4920.000000,-5303.000000,-229.000000
8319,HUL,0 days 01:16:57.197000,0 days 00:18:06.174000,10918.000000,67.222276,-21.825390,-4.299251e-04,7,0.000000,1,24617.096111,4996.291378,-5325.814788,-229.675746
8320,HUL,0 days 01:16:57.254000,0 days 00:18:06.231000,10810.107264,65.978229,-21.825414,-2.116404e-03,7,0.000000,1,24620.698600,5031.000000,-5335.000000,-230.000000
8321,HUL,0 days 01:16:57.474000,0 days 00:18:06.451000,10393.678693,61.176638,-21.825880,2.414180e+03,7,0.000000,1,24634.031800,5157.000000,-5347.000000,-231.000000
8322,HUL,0 days 01:16:57.477000,0 days 00:18:06.454000,10388.000000,61.111160,-14.583341,-4.077677e-05,6,0.000000,1,24634.207222,5159.323011,-5346.748478,-231.019124
8323,HUL,0 days 01:16:57.714000,0 days 00:18:06.691000,10249.947537,57.654908,-14.583351,1.846179e+01,6,0.000000,1,24647.527895,5374.000000,-5301.000000,-233.000000


Exploratory Data Analysis - Nico Hulkenberg

In [27]:
hulkenberg_eda_summary = f1_pandas_helpers.get_driver_eda_summary(hulkenberg_sector3_telemetry_features, hulkenberg)
hulkenberg_eda_summary

--- EDA Summary for HUL ---
Row Count: 231

Speed (m/s) -->
Max   : 87.500070
Mean  : 56.760502
Median: 57.177824
StdDev: 14.819485

Acceleration (m/s²) -->
Max   : 20.833350
Mean  : -0.831050
Median: 2.777777
StdDev: 12.631598

nGear -->
Shifts: 16

Throttle (%) -->
Events: 6
Mean  : 66.572626
StdDev: 42.205137

BrakesApplied -->
Events: 4



Q2 data for Kevin Magnussen, Haas

In [28]:
q1, q2, q3 = session.get_laps(magnussen).split_qualifying_sessions()
magnussen_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:55:34.175000"
sector_1_end = "0 days 00:55:51.220000"
sector_2_end = "0 days 00:56:27.265000"
sector_3_end = "0 days 00:56:57.907000"

# magnussen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_start, end=sector_1_end)
# magnussen_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector1_telemetry, magnussen)
# magnussen_sector1_telemetry_cleaned

# magnussen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_end, end=sector_2_end)
# magnussen_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector2_telemetry, magnussen)
# magnussen_sector2_telemetry_cleaned

magnussen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_2_end, end=sector_3_end)
magnussen_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector3_telemetry, magnussen)
magnussen_sector3_telemetry_features = feature_engineering.TelemetryFeatures(magnussen_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
magnussen_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
9959,MAG,0 days 00:56:27.314000,0 days 00:22:02.860000,11253.250112,86.504411,-23.437528,-7.444555e+00,8,49.190645,0,24578.025409,4407.000000,-5103.000000,-231.000000
9960,MAG,0 days 00:56:27.473000,0 days 00:22:03.019000,10975.000000,82.777844,-24.621212,-2.124190e-04,8,0.000000,1,24590.761667,4532.111598,-5151.881176,-230.382195
9961,MAG,0 days 00:56:27.554000,0 days 00:22:03.100000,10841.718287,80.783526,-24.621230,-4.908162e-05,8,0.000000,1,24596.800564,4593.000000,-5176.000000,-230.000000
9962,MAG,0 days 00:56:28.255000,0 days 00:22:03.801000,9688.254756,63.524044,-24.621264,5.889824e+01,8,0.000000,1,24639.063705,5046.000000,-5338.000000,-230.000000
9963,MAG,0 days 00:56:28.353000,0 days 00:22:03.899000,9527.000000,61.111160,-18.849236,2.181471e-04,6,0.000000,1,24644.539444,5100.230509,-5344.861873,-230.330333
9964,MAG,0 days 00:56:28.515000,0 days 00:22:04.061000,9498.071406,58.057584,-18.849201,1.597390e+02,6,0.000000,1,24653.580289,5188.000000,-5345.000000,-231.000000
9965,MAG,0 days 00:56:28.633000,0 days 00:22:04.179000,9477.000000,55.833378,0.000000,0.000000e+00,6,0.000000,1,24660.172778,5253.413724,-5335.503567,-231.571052
9966,MAG,0 days 00:56:28.694000,0 days 00:22:04.240000,9706.969759,55.833378,0.000000,-3.885782e+01,6,0.000000,1,24663.587175,5287.000000,-5328.000000,-232.000000
9967,MAG,0 days 00:56:28.833000,0 days 00:22:04.379000,10231.000000,55.833378,-5.401236,-2.858206e-05,5,0.000000,1,24671.339444,5361.719941,-5305.249602,-233.490554
9968,MAG,0 days 00:56:28.974000,0 days 00:22:04.520000,10111.150054,55.071804,-5.401240,-2.290138e+01,5,0.000000,1,24679.057918,5434.000000,-5275.000000,-235.000000


Exploratory Data Analysis - Kevin Magnussen

In [29]:
magnussen_eda_summary = f1_pandas_helpers.get_driver_eda_summary(magnussen_sector3_telemetry_features, magnussen)
magnussen_eda_summary

--- EDA Summary for MAG ---
Row Count: 228

Speed (m/s) -->
Max   : 86.504411
Mean  : 55.688142
Median: 55.833378
StdDev: 14.518212

Acceleration (m/s²) -->
Max   : 20.833350
Mean  : 0.231275
Median: 2.777778
StdDev: 10.995571

nGear -->
Shifts: 13

Throttle (%) -->
Events: 4
Mean  : 65.857042
StdDev: 40.119396

BrakesApplied -->
Events: 3



Q2 data for Liam Lawson, Visa CashApp

In [30]:
q1, q2, q3 = session.get_laps(lawson).split_qualifying_sessions()
lawson_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:38.695000"
sector_1_end = "0 days 00:57:55.568000"
sector_2_end = "0 days 00:58:31.626000"
sector_3_end = "0 days 00:59:01.907000"

# lawson_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_start, end=sector_1_end)
# lawson_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector1_telemetry, lawson)
# lawson_sector1_telemetry_cleaned

# lawson_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_end, end=sector_2_end)
# lawson_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector2_telemetry, lawson)
# lawson_sector2_telemetry_cleaned

lawson_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_2_end, end=sector_3_end)
lawson_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector3_telemetry, lawson)
lawson_sector3_telemetry_features = feature_engineering.TelemetryFeatures(lawson_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
lawson_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
10726,LAW,0 days 00:58:31.714000,0 days 00:23:46.395000,10792.000000,89.166738,-33.179039,-8.080131e-05,8,3.000000,1,24496.917500,4420.113095,-5108.579088,-229.972149
10727,LAW,0 days 00:58:31.834000,0 days 00:23:46.515000,10511.000000,85.185253,-33.179049,5.059965e-04,8,2.000000,1,24506.857747,4516.000000,-5146.000000,-230.000000
10728,LAW,0 days 00:58:32.053000,0 days 00:23:46.734000,9998.174850,77.919042,-33.178938,-1.974991e+02,8,0.174999,1,24523.242133,4677.000000,-5209.000000,-229.000000
10729,LAW,0 days 00:58:32.074000,0 days 00:23:46.755000,9949.000000,77.222284,-37.326419,3.157968e-13,8,0.000000,1,24524.717500,4691.704932,-5214.756971,-228.955902
10730,LAW,0 days 00:58:32.254000,0 days 00:23:46.935000,10019.875000,70.503529,-37.326419,1.949682e+02,8,0.000000,1,24536.702452,4812.000000,-5262.000000,-229.000000
10731,LAW,0 days 00:58:32.394000,0 days 00:23:47.075000,10075.000000,65.277830,-10.030872,2.930989e-12,7,0.000000,1,24545.606389,4897.183115,-5295.293931,-228.896543
10732,LAW,0 days 00:58:32.434000,0 days 00:23:47.115000,10134.333333,64.876595,-10.030872,2.516506e-13,7,0.000000,1,24548.135833,4923.000000,-5304.000000,-229.000000
10733,LAW,0 days 00:58:32.674000,0 days 00:23:47.355000,10490.333333,62.469186,-10.030872,-1.133296e+02,7,0.000000,1,24563.074221,5091.000000,-5344.000000,-231.000000
10734,LAW,0 days 00:58:32.754000,0 days 00:23:47.435000,10609.000000,61.666716,-19.097238,-3.907985e-13,5,0.000000,1,24567.806389,5136.339297,-5348.847978,-231.431531
10735,LAW,0 days 00:58:33.054000,0 days 00:23:47.735000,9681.812500,55.937545,-19.097238,9.548619e+02,5,0.000000,1,24584.474416,5267.000000,-5333.000000,-232.000000


Exploratory Data Analysis - Liam Lawson

In [31]:
lawson_eda_summary = f1_pandas_helpers.get_driver_eda_summary(lawson_sector3_telemetry_features, lawson)
lawson_eda_summary

--- EDA Summary for LAW ---
Row Count: 219

Speed (m/s) -->
Max   : 89.166738
Mean  : 56.354969
Median: 55.937545
StdDev: 14.263416

Acceleration (m/s²) -->
Max   : 29.513912
Mean  : -0.522532
Median: 1.984129
StdDev: 12.031260

nGear -->
Shifts: 13

Throttle (%) -->
Events: 4
Mean  : 65.453131
StdDev: 42.920863

BrakesApplied -->
Events: 3



Q2 data for Yuki Tsunoda, Visa CashApp

In [32]:
q1, q2, q3 = session.get_laps(tsunoda).split_qualifying_sessions()
tsunoda_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:26.587000"
sector_1_end = "0 days 00:57:43.456000"
sector_2_end = "0 days 00:58:19.489000"
sector_3_end = "0 days 00:58:49.753000"

# tsunoda_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_start, end=sector_1_end)
# tsunoda_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector1_telemetry, tsunoda)
# tsunoda_sector1_telemetry_cleaned

# tsunoda_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_end, end=sector_2_end)
# tsunoda_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector2_telemetry, tsunoda)
# tsunoda_sector2_telemetry_cleaned

tsunoda_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_2_end, end=sector_3_end)
tsunoda_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector3_telemetry, tsunoda)
tsunoda_sector3_telemetry_features = feature_engineering.TelemetryFeatures(tsunoda_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
tsunoda_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
10761,TSU,0 days 00:58:19.534000,0 days 00:23:49.717000,11323.500000,88.090348,-19.097237,-1.446760e+02,8,100.000000,0,24492.829555,4398.000000,-5100.000000,-230.000000
10762,TSU,0 days 00:58:19.594000,0 days 00:23:49.777000,11271.000000,86.944514,-27.777800,0.000000e+00,8,100.000000,1,24497.957778,4447.038251,-5119.153961,-229.912637
10763,TSU,0 days 00:58:19.774000,0 days 00:23:49.957000,11099.250000,81.944510,-27.777800,5.787042e+01,8,25.000000,1,24512.556607,4590.000000,-5175.000000,-230.000000
10764,TSU,0 days 00:58:19.834000,0 days 00:23:50.017000,11042.000000,80.277842,-24.305575,1.243450e-12,8,0.000000,1,24517.224444,4635.259050,-5192.596619,-229.662427
10765,TSU,0 days 00:58:19.934000,0 days 00:23:50.117000,11202.833333,77.847285,-24.305575,3.968298e+01,8,0.000000,1,24524.824006,4708.000000,-5221.000000,-229.000000
10766,TSU,0 days 00:58:20.074000,0 days 00:23:50.257000,11428.000000,74.444504,-18.749958,-3.600212e-03,8,0.000000,1,24535.091111,4804.833862,-5259.479019,-228.959847
10767,TSU,0 days 00:58:20.095000,0 days 00:23:50.278000,11432.672486,74.050755,-18.750033,1.873603e-04,8,0.000000,1,24536.594500,4819.000000,-5265.000000,-229.000000
10768,TSU,0 days 00:58:20.355000,0 days 00:23:50.538000,11490.522543,69.175746,-18.749985,-1.058655e+02,8,0.000000,1,24554.380780,4985.000000,-5324.000000,-230.000000
10769,TSU,0 days 00:58:20.474000,0 days 00:23:50.657000,11517.000000,66.944498,-31.347981,-1.797734e-04,5,0.000000,1,24561.868889,5052.345687,-5340.461109,-230.731588
10770,TSU,0 days 00:58:20.514000,0 days 00:23:50.697000,11547.094038,65.690579,-31.347988,7.680280e+01,5,0.000000,1,24564.282769,5079.000000,-5343.000000,-231.000000


Exploratory Data Analysis - Yuki Tsunoda

In [33]:
tsunoda_eda_summary = f1_pandas_helpers.get_driver_eda_summary(tsunoda_sector3_telemetry_features, tsunoda)
tsunoda_eda_summary

--- EDA Summary for TSU ---
Row Count: 216

Speed (m/s) -->
Max   : 88.090348
Mean  : 55.993787
Median: 56.316017
StdDev: 14.536680

Acceleration (m/s²) -->
Max   : 15.625013
Mean  : -0.791875
Median: 3.224208
StdDev: 11.902548

nGear -->
Shifts: 16

Throttle (%) -->
Events: 4
Mean  : 67.910620
StdDev: 40.055991

BrakesApplied -->
Events: 4



Q3 data for George Russell, Mercedes

In [34]:
q1, q2, q3 = session.get_laps(russell).split_qualifying_sessions()
russell_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:17:14.677000"
sector_1_end = "0 days 01:17:31.522000"
sector_2_end = "0 days 01:18:07.527000"
sector_3_end = "0 days 01:18:37.566000"

# russell_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_start, end=sector_1_end)
# russell_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector1_telemetry, russell)
# russell_sector1_telemetry_cleaned

# russell_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_end, end=sector_2_end)
# russell_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector2_telemetry, russell)
# russell_sector2_telemetry_cleaned

russell_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_2_end, end=sector_3_end)
russell_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector3_telemetry, russell)
russell_sector3_telemetry_features = feature_engineering.TelemetryFeatures(russell_sector3_telemetry_cleaned).acceleration().jerk().get_features_df()
russell_sector3_telemetry_features

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),Acceleration (m/s²),Jerk (m/s³),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
8568,RUS,0 days 01:18:07.634000,0 days 00:18:36.620000,10954.520000,88.405626,-4.166670,-3.362012e+02,8,42.000000,0,24487.916409,4435.000000,-5114.000000,-230.000000
8569,RUS,0 days 01:18:07.718000,0 days 00:18:36.704000,10637.000000,88.055626,-32.407572,1.007694e-02,8,0.000000,1,24495.208333,4502.368962,-5140.440041,-230.030414
8570,RUS,0 days 01:18:07.733000,0 days 00:18:36.719000,10657.687588,87.569512,-32.407420,-8.001527e-05,8,0.000000,1,24496.481759,4514.000000,-5145.000000,-230.000000
8571,RUS,0 days 01:18:07.894000,0 days 00:18:36.880000,10879.733333,82.351918,-32.407433,-1.808451e+02,8,0.000000,1,24509.588025,4634.000000,-5192.000000,-229.000000
8572,RUS,0 days 01:18:07.958000,0 days 00:18:36.944000,10968.000000,80.277842,-43.981517,1.149407e-12,8,0.000000,1,24514.475000,4679.963462,-5210.051940,-228.860375
8573,RUS,0 days 01:18:08.094000,0 days 00:18:37.080000,11360.133333,74.296356,-43.981517,1.669339e+02,8,0.000000,1,24524.215457,4774.000000,-5247.000000,-229.000000
8574,RUS,0 days 01:18:08.198000,0 days 00:18:37.184000,11660.000000,69.722278,-26.620392,-2.049643e-13,8,0.000000,1,24531.208333,4841.444833,-5273.332562,-228.925634
8575,RUS,0 days 01:18:08.354000,0 days 00:18:37.340000,11098.400000,65.569497,-26.620392,2.480161e+02,8,0.000000,1,24541.152930,4936.000000,-5308.000000,-229.000000
8576,RUS,0 days 01:18:08.438000,0 days 00:18:37.424000,10796.000000,63.333384,-5.787042,9.349247e-13,6,0.000000,1,24546.408333,4984.147576,-5323.157756,-229.498407
8577,RUS,0 days 01:18:08.514000,0 days 00:18:37.500000,10933.116667,62.893569,-5.787042,-7.964744e+01,6,0.000000,1,24551.171420,5027.000000,-5334.000000,-230.000000


Exploratory Data Analysis - George Russell

In [35]:
russell_eda_summary = f1_pandas_helpers.get_driver_eda_summary(russell_sector3_telemetry_features, russell)
russell_eda_summary

--- EDA Summary for RUS ---
Row Count: 221

Speed (m/s) -->
Max   : 88.405626
Mean  : 56.934173
Median: 56.666712
StdDev: 15.136357

Acceleration (m/s²) -->
Max   : 20.833359
Mean  : -1.240075
Median: 2.314817
StdDev: 12.696616

nGear -->
Shifts: 14

Throttle (%) -->
Events: 5
Mean  : 65.214135
StdDev: 41.125276

BrakesApplied -->
Events: 3



Sector 1 telemetry data for all drivers is concatanated into a single dataframe below.

In [36]:
# total_sector1_telemetry = pd.concat([
#     norris_sector1_telemetry_cleaned, 
#     piastri_sector1_telemetry_cleaned, 
#     verstappen_sector1_telemetry_cleaned, 
#     perez_sector1_telemetry_cleaned,
#     sainz_sector1_telemetry_cleaned,
#     leclerc_sector1_telemetry_cleaned,
#     bottas_sector1_telemetry_cleaned,
#     alonso_sector1_telemetry_cleaned,
#     stroll_sector1_telemetry_cleaned,
#     gasly_sector1_telemetry_cleaned,
#     hulkenberg_sector1_telemetry_cleaned,
#     magnussen_sector1_telemetry_cleaned,
#     lawson_sector1_telemetry_cleaned,
#     tsunoda_sector1_telemetry_cleaned,
#     russell_sector1_telemetry_cleaned
# ], axis=0)

# total_sector1_telemetry

Sector 2 telemetry data for all drivers is concatenated into a single dataframe below.

In [37]:
# total_sector2_telemetry = pd.concat([
#     norris_sector2_telemetry_cleaned, 
#     piastri_sector2_telemetry_cleaned, 
#     verstappen_sector2_telemetry_cleaned, 
#     perez_sector2_telemetry_cleaned,
#     sainz_sector2_telemetry_cleaned,
#     leclerc_sector2_telemetry_cleaned,
#     bottas_sector2_telemetry_cleaned,
#     alonso_sector2_telemetry_cleaned,
#     stroll_sector2_telemetry_cleaned,
#     gasly_sector2_telemetry_cleaned,
#     hulkenberg_sector2_telemetry_cleaned,
#     magnussen_sector2_telemetry_cleaned,
#     lawson_sector2_telemetry_cleaned,
#     tsunoda_sector2_telemetry_cleaned,
#     russell_sector2_telemetry_cleaned
# ], axis=0)

# total_sector2_telemetry

Sector 3 telemetry data for all drivers is concatanated into a single dataframe below.

In [38]:
# total_sector3_telemetry = pd.concat([
#     norris_sector3_telemetry_cleaned, 
#     piastri_sector3_telemetry_cleaned, 
#     verstappen_sector3_telemetry_cleaned, 
#     perez_sector3_telemetry_cleaned,
#     sainz_sector3_telemetry_cleaned,
#     leclerc_sector3_telemetry_cleaned,
#     bottas_sector3_telemetry_cleaned,
#     alonso_sector3_telemetry_cleaned,
#     stroll_sector3_telemetry_cleaned,
#     gasly_sector3_telemetry_cleaned,
#     hulkenberg_sector3_telemetry_cleaned,
#     magnussen_sector3_telemetry_cleaned,
#     lawson_sector3_telemetry_cleaned,
#     tsunoda_sector3_telemetry_cleaned,
#     russell_sector3_telemetry_cleaned
# ], axis=0)

# total_sector3_telemetry